In [1]:
import nltk
import pandas as pd
from pandas import DataFrame
import requests
import json
from pymongo import MongoClient
import re
import string
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import pickle
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer, MWETokenizer
from nltk.util import ngrams
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.corpus import stopwords, wordnet, words


from sklearn.preprocessing import scale, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

from gensim import corpora, models, similarities, matutils, utils
import gensim

import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import warnings
pd.set_option('display.max_columns', 101)
pd.set_option('display.max_colwidth', -1)

In [39]:
#!ls 

## Cleaning The Data

In [2]:
data_review = pd.read_json('yelp_academic_dataset_review.json', lines=True)
data_review= data_review.rename(index=str, columns={"text": "reviews", "stars": "stars_reviews"})
data_review.head(3)

,business_id,date,review_id,stars_reviews,reviews,type,user_id,votes
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,"My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best ""toast"" I've ever had.\n\nAnyway, I can't wait to go back!",review,rLtl8ZkDX5vH5nAx9C3q5Q,"{'funny': 0, 'useful': 5, 'cool': 2}"
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,"I have no idea why some people give bad reviews about this place. It goes to show you, you can please everyone. They are probably griping about something that their own fault...there are many people like that.\n\nIn any case, my friend and I arrived at about 5:50 PM this past Sunday. It was pretty crowded, more than I thought for a Sunday evening and thought we would have to wait forever to get a seat but they said we'll be seated when the girl comes back from seating someone else. We were seated at 5:52 and the waiter came and got our drink orders. Everyone was very pleasant from the host that seated us to the waiter to the server. The prices were very good as well. We placed our orders once we decided what we wanted at 6:02. We shared the baked spaghetti calzone and the small ""Here's The Beef"" pizza so we can both try them. The calzone was huge and we got the smallest one (personal) and got the small 11"" pizza. Both were awesome! My friend liked the pizza better and I liked the calzone better. The calzone does have a sweetish sauce but that's how I like my sauce!\n\nWe had to box part of the pizza to take it home and we were out the door by 6:42. So, everything was great and not like these bad reviewers. That goes to show you that you have to try these things yourself because all these bad reviewers have some serious issues.",review,0a2KyEL0d3Yb1V6aivbIuQ,"{'funny': 0, 'useful': 0, 'cool': 0}"
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I also dig their candy selection :),review,0hT2KtfLiobPvh6cDC8JQg,"{'funny': 0, 'useful': 1, 'cool': 0}"


In [ ]:
#data_user = pd.read_json('yelp_academic_dataset_user.json', lines=True)
#data_user.head()

In [ ]:
#data_ch = pd.read_json('yelp_academic_dataset_checkin.json', lines=True)
#data_ch.head()

In [3]:
data_business = pd.read_json('business.json', lines=True)
data_business.head(3)

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,rncjoVoEFUJGCUoC1JgnUA,"[Accountants, Professional Services, Tax Services, Financial Services]",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business
1,0FNFSzCFP_rGUoJx8W7tJg,"[Sporting Goods, Bikes, Shopping]",Phoenix,"2149 W Wood Dr\nPhoenix, AZ 85029",33.604054,-112.105933,Bike Doctor,[],True,5,5.0,AZ,business
2,3f_lyB6vFK48ukH6ScvLHg,[],Phoenix,"1134 N Central Ave\nPhoenix, AZ 85004",33.460526,-112.073933,Valley Permaculture Alliance,[],True,4,5.0,AZ,business


In [46]:
df_merge = pd.merge(data_review, data_business, on='business_id')
df_merge 

,business_id,date,review_id,stars_reviews,reviews,type_x,user_id,votes,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type_y
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,"My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best ""toast"" I've ever had.\n\nAnyway, I can't wait to go back!",review,rLtl8ZkDX5vH5nAx9C3q5Q,"{'funny': 0, 'useful': 5, 'cool': 2}","[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,[],True,116,4.0,AZ,business
1,9yKzy9PApeiPPOUJEtnvkg,2011-04-20,4nvfVCoIDBhEiQdvXh1xoA,2,"Finally was able to make it here for breakfast. My opinion, I will NOT be back. As we were walking up to the place, I said ""O now this is the kind of place we should be taking our out of town visitors."". Well, on 2nd thought, NO. We went today, a weds, there was no wait and we were seated promptly. We were informed that the beverages was self serve as far as coffee, water, tea and lemonade goes. We started with tea-dislike, then lemonade-dislike. And did I mention we self served ourselves in a 12 oz cup with a price of $2.75, WOW! If it was to our liking, Id say that is still steep for a self serve drink in a 12oz cup. He had the monte cristo, which he liked but was not blown away for the price. I had the french toast and it was only ok. I also ordered a side of sausage that i DISLIKED and it was $4 for I think 2 links, WOW. And once our food came, the flys were EVERYHWERE. I kid you not when I say I couldnt wait to get outta there. I left very disappointed, and hungry. Will never return or recommend. But hey, try it for yourself, you may like it. I just think it was too high priced for what we got.",review,gXmtPKLWPZJeJX_KPw54HA,"{'funny': 0, 'useful': 2, 'cool': 0}","[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,[],True,116,4.0,AZ,business
2,9yKzy9PApeiPPOUJEtnvkg,2012-05-08,VQqwoz4s-xYwGoqJDASTfg,2,":( I really wanted to like this place. I thoroughly enjoyed the farm, but I will review it separately. We were greeted by a busy waiter who quickly pointed to self-serve drinks (coffee, tea, water and lemonade) and ran off. We helped ourselves to some coffee and lemonade before we received our menus which stated that each drink was $2.75. I ordered the white truffled scrambled egg skillet (white truffles are my fave). I did not detect one bit of truffle essence in this dish. My fiance ordered the Cowboy Breakfast. It was like watered down chili with and egg on top. No great. The concept is great, but the way in which the food is prepared is super disappointing.",review,BvjSQAFcROLp27QjaRcyoA,"{'funny': 1, 'useful': 1, 'cool': 0}","[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,[],True,116,4.0,AZ,business
3,9yKzy9PApeiPPOUJEtnvkg,2012-06-03,PiR7lltpkA4GP8gAeCJYIg,4,"Loved the scenic restaurant, right in the middle of the farm. My #1 spot for a weekend brunch.",review,5qa1hx5GVHehlBQx0b5gFw,"{'funny': 0, 'useful': 0, 'cool': 0}","[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 850

In [47]:
df_merge_2 = df_merge[["stars_reviews", "reviews", 'user_id', "categories", "city", "full_address", "latitude", "longitude", "name", "review_count", "stars", "state"]]

In [48]:
df_merge_2.head(3)

,stars_reviews,reviews,user_id,categories,city,full_address,latitude,longitude,name,review_count,stars,state
0,5,"My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best ""toast"" I've ever had.\n\nAnyway, I can't wait to go back!",rLtl8ZkDX5vH5nAx9C3q5Q,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ
1,2,"Finally was able to make it here for breakfast. My opinion, I will NOT be back. As we were walking up to the place, I said ""O now this is the kind of place we should be taking our out of town visitors."". Well, on 2nd thought, NO. We went today, a weds, there was no wait and we were seated promptly. We were informed that the beverages was self serve as far as coffee, water, tea and lemonade goes. We started with tea-dislike, then lemonade-dislike. And did I mention we self served ourselves in a 12 oz cup with a price of $2.75, WOW! If it was to our liking, Id say that is still steep for a self serve drink in a 12oz cup. He had the monte cristo, which he liked but was not blown away for the price. I had the french toast and it was only ok. I also ordered a side of sausage that i DISLIKED and it was $4 for I think 2 links, WOW. And once our food came, the flys were EVERYHWERE. I kid you not when I say I couldnt wait to get outta there. I left very disappointed, and hungry. Will never return or recommend. But hey, try it for yourself, you may like it. I just think it was too high priced for what we got.",gXmtPKLWPZJeJX_KPw54HA,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ
2,2,":( I really wanted to like this place. I thoroughly enjoyed the farm, but I will review it separately. We were greeted by a busy waiter who quickly pointed to self-serve drinks (coffee, tea, water and lemonade) and ran off. We helped ourselves to some coffee and lemonade before we received our menus which stated that each drink was $2.75. I ordered the white truffled scrambled egg skillet (white truffles are my fave). I did not detect one bit of truffle essence in this dish. My fiance ordered the Cowboy Breakfast. It was like watered down chili with and egg on top. No great. The concept is great, but the way in which the food is prepared is super disappointing.",BvjSQAFcROLp27QjaRcyoA,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ


In [49]:
df_merge_3 = df_merge_2[['categories']].join(df_merge_2.categories.str.join('|').str.get_dummies().add_prefix(''))

In [50]:
df_merge_3.shape

(229907, 509)

In [51]:
df_merge_4 = df_merge_3[["Restaurants"]]
df_merge_rest = pd.concat([df_merge_2, df_merge_4.reindex(df_merge_2.index)], axis=1)


In [53]:
df_merge_rest.shape

(229907, 13)

In [54]:
df_merge_rest2 = df_merge_rest[df_merge_rest['Restaurants'] == 1]
df_merge_rest2.head()

,stars_reviews,reviews,user_id,categories,city,full_address,latitude,longitude,name,review_count,stars,state,Restaurants
0,5,"My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best ""toast"" I've ever had.\n\nAnyway, I can't wait to go back!",rLtl8ZkDX5vH5nAx9C3q5Q,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
1,2,"Finally was able to make it here for breakfast. My opinion, I will NOT be back. As we were walking up to the place, I said ""O now this is the kind of place we should be taking our out of town visitors."". Well, on 2nd thought, NO. We went today, a weds, there was no wait and we were seated promptly. We were informed that the beverages was self serve as far as coffee, water, tea and lemonade goes. We started with tea-dislike, then lemonade-dislike. And did I mention we self served ourselves in a 12 oz cup with a price of $2.75, WOW! If it was to our liking, Id say that is still steep for a self serve drink in a 12oz cup. He had the monte cristo, which he liked but was not blown away for the price. I had the french toast and it was only ok. I also ordered a side of sausage that i DISLIKED and it was $4 for I think 2 links, WOW. And once our food came, the flys were EVERYHWERE. I kid you not when I say I couldnt wait to get outta there. I left very disappointed, and hungry. Will never return or recommend. But hey, try it for yourself, you may like it. I just think it was too high priced for what we got.",gXmtPKLWPZJeJX_KPw54HA,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
2,2,":( I really wanted to like this place. I thoroughly enjoyed the farm, but I will review it separately. We were greeted by a busy waiter who quickly pointed to self-serve drinks (coffee, tea, water and lemonade) and ran off. We helped ourselves to some coffee and lemonade before we received our menus which stated that each drink was $2.75. I ordered the white truffled scrambled egg skillet (white truffles are my fave). I did not detect one bit of truffle essence in this dish. My fiance ordered the Cowboy Breakfast. It was like watered down chili with and egg on top. No great. The concept is great, but the way in which the food is prepared is super disappointing.",BvjSQAFcROLp27QjaRcyoA,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
3,4,"Loved the scenic restaurant, right in the middle of the farm. My #1 spot for a weekend brunch.",5qa1hx5GVHehlBQx0b5gFw,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
4,5,Beautiful country setting in Phoenix on an old farm. Watch closely because it is easy to miss as it is actually an old farm near the Legacy Golf resort at South Mountain. \nVery good home made breakfast with fresh ingredients from the farm. \nOrange juice was fresh and tasty. \nEggs and bacon were very good. Biscuits and gravy were excellent. \nThe setting was the star of this breakfast!\nI would recommend ! You won't be sorry !,nprSBcvBhvzyIbac

In [55]:
df_merge_rest_AZ = df_merge_rest2[df_merge_rest2["state"] == 'AZ']
df_merge_rest_AZ.shape


(158425, 13)

In [54]:
df_merge_rest2 = df_merge_rest[df_merge_rest['Restaurants'] == 1]
df_merge_rest2.head()

,stars_reviews,reviews,user_id,categories,city,full_address,latitude,longitude,name,review_count,stars,state,Restaurants
0,5,"My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I've ever had. I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best ""toast"" I've ever had.\n\nAnyway, I can't wait to go back!",rLtl8ZkDX5vH5nAx9C3q5Q,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
1,2,"Finally was able to make it here for breakfast. My opinion, I will NOT be back. As we were walking up to the place, I said ""O now this is the kind of place we should be taking our out of town visitors."". Well, on 2nd thought, NO. We went today, a weds, there was no wait and we were seated promptly. We were informed that the beverages was self serve as far as coffee, water, tea and lemonade goes. We started with tea-dislike, then lemonade-dislike. And did I mention we self served ourselves in a 12 oz cup with a price of $2.75, WOW! If it was to our liking, Id say that is still steep for a self serve drink in a 12oz cup. He had the monte cristo, which he liked but was not blown away for the price. I had the french toast and it was only ok. I also ordered a side of sausage that i DISLIKED and it was $4 for I think 2 links, WOW. And once our food came, the flys were EVERYHWERE. I kid you not when I say I couldnt wait to get outta there. I left very disappointed, and hungry. Will never return or recommend. But hey, try it for yourself, you may like it. I just think it was too high priced for what we got.",gXmtPKLWPZJeJX_KPw54HA,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
2,2,":( I really wanted to like this place. I thoroughly enjoyed the farm, but I will review it separately. We were greeted by a busy waiter who quickly pointed to self-serve drinks (coffee, tea, water and lemonade) and ran off. We helped ourselves to some coffee and lemonade before we received our menus which stated that each drink was $2.75. I ordered the white truffled scrambled egg skillet (white truffles are my fave). I did not detect one bit of truffle essence in this dish. My fiance ordered the Cowboy Breakfast. It was like watered down chili with and egg on top. No great. The concept is great, but the way in which the food is prepared is super disappointing.",BvjSQAFcROLp27QjaRcyoA,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
3,4,"Loved the scenic restaurant, right in the middle of the farm. My #1 spot for a weekend brunch.",5qa1hx5GVHehlBQx0b5gFw,"[Breakfast & Brunch, Restaurants]",Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",33.390792,-112.012504,Morning Glory Cafe,116,4.0,AZ,1
4,5,Beautiful country setting in Phoenix on an old farm. Watch closely because it is easy to miss as it is actually an old farm near the Legacy Golf resort at South Mountain. \nVery good home made breakfast with fresh ingredients from the farm. \nOrange juice was fresh and tasty. \nEggs and bacon were very good. Biscuits and gravy were excellent. \nThe setting was the star of this breakfast!\nI would recommend ! You won't be sorry !,nprSBcvBhvzyIbac

# To MongoDB

In [60]:
import json
import pandas as pd
import pymongo
from pymongo import MongoClient
from pprint import pprint
client=MongoClient()


In [61]:
db=client.datascience # datascience is my database name(even if you dont have one, it creates byitself)


In [62]:
df_mongo=df_merge_rest_AZ.to_dict('record') # creating dictionary file==>data_unique is my pandas table file name


In [63]:
db.df_mongo.insert_many(df_mongo) # putting all your list of dictionary into your mongodb


In [64]:
list(db.df_mongo.find({}))
cursor=db.df_mongo.find({},{'city':'Phoenix'})
list(cursor)

[{'_id': ObjectId('5dcd9e37122175d358209fb9'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fba'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fbb'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fbc'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fbd'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fbe'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fbf'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc0'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc1'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc2'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc3'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc4'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc5'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc6'), 'city': 'Phoenix'},
 {'_id': ObjectId('5dcd9e37122175d358209fc7'), 'city': 'Phoeni

In [59]:
# Save dataframe into pickle to be used later. This helps to skip repeating the data import and conversion process.
with open('df_merge_rest_AZ2.pickle', 'wb') as to_write:
    pickle.dump(df_merge_rest_AZ, to_write)